In [7]:
import os
from google.colab import drive
drive.mount("/trainPNGSeg/")
os.chdir("/trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg")

Drive already mounted at /trainPNGSeg/; to attempt to forcibly remount, call drive.mount("/trainPNGSeg/", force_remount=True).


In [12]:
%ls

'='/  ')'/   1/   4/   7/   div/   gt/             lt/    pm/
 -/    +/    2/   5/   8/   e/     label_map.npy   neq/   times/
'('/   0/    3/   6/   9/   geq/   leq/            pi/


In [18]:
from os.path import isfile
if isfile("label_map.npy"):
    class_indices = np.load("label_map.npy",allow_pickle=True).item()
print(class_indices)
print(len(class_indices))

{'(': 0, ')': 1, '+': 2, '-': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '=': 14, 'div': 15, 'e': 16, 'geq': 17, 'gt': 18, 'leq': 19, 'lt': 20, 'neq': 21, 'pi': 22, 'pm': 23, 'times': 24}
25


In [19]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


# generate training data-75% and validation data-25%
train_data_dir = "/trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg"
data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.25)

train_dataset = data_gen.flow_from_directory(train_data_dir, target_size=(32, 32), color_mode="grayscale", batch_size= 15, subset='training')
validation_dataset = data_gen.flow_from_directory(train_data_dir, target_size=(32, 32), color_mode="grayscale", batch_size= 15, subset='validation')

# numClasses = how many characters total in label_map
label_map = (train_dataset.class_indices)
numClasses = len(label_map)

# save tge label_map to local as .npy
np.save('label_map_v3.npy', label_map)

# overwrite and save the model with the best val_acc (the accuracy of a batch of testing data)
checkpoint = ModelCheckpoint(train_data_dir, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# initialize lenet model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), padding='same', input_shape = (32,32,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=48, kernel_size=(5,5), strides=(1,1), padding='valid', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(units=numClasses, activation = 'softmax'))

# compile model & fit model with training and validation data
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("training...")

model.fit(train_dataset, steps_per_epoch = len(train_dataset), validation_data = validation_dataset,  validation_steps = len(validation_dataset), epochs = 15, callbacks = [checkpoint])

#saving model to .h5 file in train_data_dir
print("saving model...")
model.save('LeNetModel_v3.h5')

print("evaluating...")
score = model.evaluate(validation_dataset)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])



Found 6922 images belonging to 25 classes.
Found 2298 images belonging to 25 classes.
training...
Epoch 1/15
462/462 [==============================] - 30s 64ms/step - loss: 2.9872 - accuracy: 0.1194 - val_loss: 1.3649 - val_accuracy: 0.5705

Epoch 00001: val_accuracy improved from -inf to 0.57050, saving model to /trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg
INFO:tensorflow:Assets written to: /trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg/assets
Epoch 2/15
462/462 [==============================] - 31s 67ms/step - loss: 1.0383 - accuracy: 0.6659 - val_loss: 0.7933 - val_accuracy: 0.7319

Epoch 00002: val_accuracy improved from 0.57050 to 0.73194, saving model to /trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg
INFO:tensorflow:Assets written to: /trainPNGSeg/Shareddrives/515_handwritten/trainPNGSeg/assets
Epoch 3/15
462/462 [==============================] - 29s 63ms/step - loss: 0.6577 - accuracy: 0.7836 - val_loss: 0.6510 - val_accuracy: 0.7763

Epoch 00003: val_a